### RAG Pipelines - Data Ingestion to Vector DB ###

In [ ]:
import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path

from dotenv import load_dotenv
load_dotenv()




### Converting Raw PDF/File to Langchain Document Structure

In [ ]:
### Read all the pdf's inside directory ###

all_documents = []
def process_documents(pdf_directory):
    """Processes all the PDF's that are present inside the given pdf_diretory"""

    pdf_dir = Path(pdf_directory)

    pdf_files = list(pdf_dir.glob("**/*.pdf"))
    print(f"Found {len(pdf_files)} PDF's to process.")
    print(f"\n\nPDF FILES = {pdf_files}\n\n")

    for pdf_file in pdf_files:
        print(f"Processing {pdf_file.name} pdf\n")

        try:

            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()

            ## Adding information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'


            all_documents.extend(documents)
            print(f"✅Loaded {len(documents)} documents")



        except Exception as e:
            print("❌Error in loading documents")

        print(f"\nTotal documents loaded: {len(documents)}")
        return documents


# Calling the above function
process_documents("../data/pdf_files/")


In [ ]:
print(all_documents)

### Splitting the Document structure to Smaller chunks/pieces

In [ ]:
# Splitting Text into Chunks

def split_documents(documents,chunk_size=1000,chunk_overlap=200):
    """Splits documents into smaller chunks for better RAG performance"""

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_overlap=chunk_overlap,
        chunk_size=chunk_size,
        length_function = len,
        separators=["\n\n","\n"," ",""]
    )

    split_docs= text_splitter.split_documents(documents)

    print(f"\nSplit {len(documents)} documents into {len(split_docs)} documents successfully")

    if split_docs:
        print(f"\nExample chunk: ")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")


    return split_docs

In [ ]:
chunks = split_documents(all_documents)
print(chunks)

### Embedding & Vector Store DB

In [ ]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any , Tuple
from sklearn.metrics.pairwise import cosine_similarity



In [ ]:
class EmbeddingsManager:
    """Handles the embedding generation using Sentence Transformers"""

    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
            Initialize the Embedding Manager
        
            Args:
            model_name: HuggingFace model name for sentence embeddings
        """

        self.model_name = model_name
        self.model = None
        self._load_model()


    def _load_model(self):
        """Load the Sentence Transformer Model"""

        try:

            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully: Embedding dimension: {self.model.get_sentence_embedding_dimension()}")


        except Exception as e:
            print(f"Error while loading {self.model_name}: {e}")
            raise


    def generate_embeddings(self,texts: List[str]) -> np.ndarray:

        """Generate embeddings for a list of texts
         
          Args: 
                    texts: List of text strings
                     
                      
                    Returns: 
                            numpy array of embeddings with shape (len(texts), embedding_dimensions) """
        
        if not self.model:
            raise ValueError("Model not loaded")
        

        print(f"Generate embeddings for {len(texts)} documents")
        embeddings = self.model.encode(texts,show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings
    

    def get_embeddings_dimensions(self):
        """Get the sentence embedding dimensions for a model"""

        if not self.model:
            raise ValueError("Model not loaded correctly")

        return self.model.get_sentence_embedding_dimension()    




In [ ]:
# Initlalize the Embedding manager

embedding_manager = EmbeddingsManager()
print(embedding_manager)

### Initializing Vector Store

In [ ]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any , Tuple
from sklearn.metrics.pairwise import cosine_similarity


class VectorStore:
    """Manages vector embeddings in a ChromaDB vector store"""

    def __init__(self, collection_name: str ="pdf_files", persistDirectory: str = "../data/vector_store"):
        
        self.collection_name = collection_name
        self.persistDirectory = persistDirectory
        self.client = None
        self.initialize_store()
        self.collection = self.client.get_or_create_collection(name=self.collection_name)


    def initialize_store(self):

        """Initializes ChromaDB client and collection """

        try:

            os.makedirs(self.persistDirectory, exist_ok=True) 
           #self.client = chromadb.PersistentClient(path=self.persistDirectory)

            self.client = chromadb.CloudClient(
               api_key=CHROMA_API_KEY,
               tenant = CHROMA_TENANT,
               database=CHROMA_DB
           )

            # Get or create a collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description":"PFDF document embeddings for RAG"}
            )

            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")

        except Exception as e:
            print(f"Error while initializing ChromaDB client: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
            Add documents and their embeddings to the vector store.
        """


        try:

            if(len(documents) != len(embeddings)):
                raise ValueError("Documents and embeddings must have the same length")

            print(f"Adding {len(documents)} documents to the Vector STore...")


            # Prepare data for ChromaDB

            ids = []
            metadatas = []
            documents_text = []
            embeddings_list = []

            for i, (doc,embedding) in enumerate(zip(documents,embeddings)):

                #Generate Unique ID
                doc_id = f"doc_{uuid.uuidv4().hex[:8]}_{i}"
                ids.append(doc_id)

                #Prepare Metadata

                metadata = dict(doc.metadata)
                metadata['doc_index'] = i
                metadata['content_length'] = len(doc.page_content)
                metadatas.append(metadata)


                #Document content
                documents_text.append(doc.page_content)

                #Embeddings
                embeddings_list.append(embedding.tolist())


            self.collection.add(
                ids = ids,
                documents = documents_text,
                embeddings = embeddings_list,
                metadatas = metadatas
            )

            print(f"Successfully added ${len(documents)} documents to ChromDB collection.")
            print(f"Total documents in collection: {self.collection.count()}")




        except Exception as e:
            print(f"Error while adding documents/embeddings to ChromaDB collection: {e}")
            raise



vectorStore = VectorStore()
vectorStore
        